# 预测2023年7月1-7日各品类蔬菜批发价

In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima.arima.utils import ndiffs
import pmdarima as pm
import warnings
from statsmodels.tsa.statespace.sarimax import SARIMAX

df =pd.read_excel("品类日批发价.xlsx",index_col=0)
df.index.freq = df.index.inferred_freq

predicted = pd.DataFrame(index=pd.date_range(start='2023-07-01', end='2023-07-08'))

In [2]:
# 检查各列数据是否具有季节性、时间序列数据是否静止
for col in df.columns:
    decompose_result = seasonal_decompose(df[col], model='additive', period=30)
    print(decompose_result.seasonal.sum()) # 非0表示具有季节性
    n_adf = ndiffs(df[col], test='adf')  # 0表示数据静止
    if n_adf == 1:
        print(col)

0.16355736375661545
0.2927349594356227
-1.3005728165784856
辣椒类
1.1203128677248653
0.11676765167548789
-0.6986260383597878


## 注意：自动寻参的结果参数在模型拟合是可能会达到训练阈值次数，但是由于各模型的预测结果在合理的范围内，我们因此接受该结果；此外，该段代码需要20左右分钟时间

In [44]:
# 使用SARIMA模型来拟合历史批发价数据，使用auto_arima自动寻找参数
for col in df.columns:
    model = pm.auto_arima(df[col], 
                          test='adf', # use adftest to find optimal 'd'
                          m=12, # frequency of series (if m==1, seasonal is set to FALSE automatically)
                          seasonal=True, # No Seasonality for standard ARIMA
                          suppress_warnings=True,
                          stepwise=True)
    # 获取不同列的模型的参数
    a,b,c=model.order
    model = SARIMAX(df[col], order=(a, b, c),
                    seasonal_order=(a, b, c, 12))
    fitted = model.fit()
    print(f"AIC: {fitted.aic}, BIC: {fitted.bic}, MSE: {fitted.mse}, MAE: {fitted.mae}, Order: {model.order}")
    predictions = fitted.predict(len(df), len(df)+7) # 预测未来一周的数据（此处多预测一天）
    predicted[col]=predictions.values #数据存储于predicted

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.03788D+00    |proj g|=  1.83885D+00

At iterate    5    f=  7.24800D-01    |proj g|=  8.55961D-02

At iterate   10    f=  6.27086D-01    |proj g|=  8.60210D-02

At iterate   15    f=  6.09569D-01    |proj g|=  4.66550D-02

At iterate   20    f=  6.06620D-01    |proj g|=  1.74293D-03

At iterate   25    f=  6.05736D-01    |proj g|=  1.09916D-02

At iterate   30    f=  6.04205D-01    |proj g|=  1.40660D-02

At iterate   35    f=  6.04051D-01    |proj g|=  8.98417D-05

At iterate   40    f=  6.04048D-01    |proj g|=  5.51917D-03

At iterate   45    f=  6.03843D-01    |proj g|=  2.29117D-02


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "



At iterate   50    f=  6.03587D-01    |proj g|=  1.15552D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    9     50     63      1     0     0   1.156D-03   6.036D-01
  F =  0.60358661351720788     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
AIC: 1335.0259906945475, BIC: 1379.9796405670452, MSE: 0.24793655559333838, MAE: 0.271471276985095, Order: (3, 0, 1)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning

...

 This problem is unconstrained.



At iterate    5    f=  1.71560D+00    |proj g|=  1.32973D-01

At iterate   10    f=  1.67901D+00    |proj g|=  1.71099D-02

At iterate   15    f=  1.67718D+00    |proj g|=  1.98045D-03

At iterate   20    f=  1.67697D+00    |proj g|=  3.91934D-03

At iterate   25    f=  1.67689D+00    |proj g|=  6.24934D-04

At iterate   30    f=  1.67688D+00    |proj g|=  2.60389D-03

At iterate   35    f=  1.67663D+00    |proj g|=  1.35805D-02

At iterate   40    f=  1.67627D+00    |proj g|=  1.48806D-03

At iterate   45    f=  1.67623D+00    |proj g|=  1.01692D-04


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "



At iterate   50    f=  1.67623D+00    |proj g|=  7.22655D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   13     50     55      1     0     0   7.227D-04   1.676D+00
  F =   1.6762312969277682     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
AIC: 3683.5366898963903, BIC: 3748.46973971222, MSE: 1.9524955579824617, MAE: 0.7355311211118802, Order: (5, 0, 1)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [35]:
predicted.to_excel('2023年7月1-7日批发价.xlsx',index=True)